In [24]:
import pickle
import pandas as pd
import math
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os


from scipy.stats import linregress

def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10


PATH_VARIABLES = "./variables"
DATA_PATH = "./DatosExperimento1"

# Importar el diccionario
with open(PATH_VARIABLES+"/voluntarios.pkl", 'rb') as archivo:
    voluntarios = pickle.load(archivo)
voluntarios

{'khleef-almutairi': {'nombre': 'Khleef',
  'apellidos': 'Almutairi',
  'edad': 31,
  'genero': 'H',
  'experto': True,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T112353.mat',
   'ResultsJNDsNoiseFlatBG20211129T120005.mat'],
  'universidad': 'UPV'},
 'juan-giner': {'nombre': 'Juan',
  'apellidos': 'Giner',
  'edad': 60,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T133717.mat',
   'ResultsJNDsNoiseFlatBG20211129T140322.mat'],
  'universidad': 'UPV'},
 'sergio-camp': {'nombre': 'Sergio',
  'apellidos': 'Camp',
  'edad': 49,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T163621.mat',
   'ResultsJNDsNoiseFlatBG20211129T171934.mat'],
  'universidad': 'UPV'},
 'antonio-hervas': {'nombre': 'Antonio',
  'apellidos': 'Hervas',
  'edad': 63,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211130T105700.mat',
   'ResultsJNDsNoiseFlatBG20220111T121738.mat'],
  'universidad': 'UPV'},

In [27]:
# TOTAL UGR + UPV agrupado por edad, genero y sexo

mean_result_agrupado = {"genero":{"H": None, "M": None}, "experiencia":{"Si": None, "No": None}, "edad": {"<= 23.0": None, "(23.0, 40.22]":None, "> 40.22":None}}
contadores = {"genero":{"H": 0, "M": 0}, "experiencia":{"Si": 0, "No": 0}, "edad": {"<= 23.0": 0, "(23.0, 40.22]":0, "> 40.22":0}}
for voluntario in voluntarios.values():
    for fichero in voluntario["data_files"]:
        universidad = voluntario["universidad"]
        edad = voluntario["edad"]
        sexo = voluntario["genero"]
        experiencia = voluntario["experto"]
        data = scipy.io.loadmat(DATA_PATH + "/" + universidad + "/" + fichero)
        result = np.array(data['Results'])
        datos = np.array(data['datos'])[0]
        
        # pendiente promediar usuario
        
    if edad <= 23:
        contadores["edad"]["<= 23.0"] = contadores["edad"]["<= 23.0"] + 1
        if mean_result_agrupado["edad"]["<= 23.0"] is None:
            mean_result_agrupado["edad"]["<= 23.0"] = result
        else:
            mean_result_agrupado["edad"]["<= 23.0"] = mean_result_agrupado["edad"]["<= 23.0"]+result
    elif edad > 40.22:
        contadores["edad"]["> 40.22"] = contadores["edad"]["> 40.22"]+1
        if mean_result_agrupado["edad"]["> 40.22"] is None:
            mean_result_agrupado["edad"]["> 40.22"] = result
        else:
            mean_result_agrupado["edad"]["> 40.22"] = mean_result_agrupado["edad"]["> 40.22"]+result
    else:
        contadores["edad"]["(23.0, 40.22]"] = contadores["edad"]["(23.0, 40.22]"]+1
        if mean_result_agrupado["edad"]["(23.0, 40.22]"] is None:
            mean_result_agrupado["edad"]["(23.0, 40.22]"] = result
        else:
            mean_result_agrupado["edad"]["(23.0, 40.22]"] = mean_result_agrupado["edad"]["(23.0, 40.22]"]+result
            
    if sexo == "H":
        contadores["genero"]["H"] = contadores["genero"]["H"]+1
        if mean_result_agrupado["genero"]["H"] is None:
            mean_result_agrupado["genero"]["H"] = result
        else:
            mean_result_agrupado["genero"]["H"] = mean_result_agrupado["genero"]["H"]+result
    else:
        contadores["genero"]["M"] = contadores["genero"]["M"]+1
        if mean_result_agrupado["genero"]["M"] is None:
            mean_result_agrupado["genero"]["M"] = result
        else:
            mean_result_agrupado["genero"]["M"] = mean_result_agrupado["genero"]["M"]+result
            
    if experiencia:
        contadores["experiencia"]["Si"] = contadores["experiencia"]["Si"]+1
        if mean_result_agrupado["experiencia"]["Si"] is None:
            mean_result_agrupado["experiencia"]["Si"] = result
        else:
            mean_result_agrupado["experiencia"]["Si"] = mean_result_agrupado["experiencia"]["Si"]+result
    else:
        contadores["experiencia"]["No"] = contadores["experiencia"]["No"]+1
        if mean_result_agrupado["experiencia"]["No"] is None:
            mean_result_agrupado["experiencia"]["No"] = result
        else:
            mean_result_agrupado["experiencia"]["No"] = mean_result_agrupado["experiencia"]["No"]+result

mean_result_agrupado["edad"]["<= 23.0"] = mean_result_agrupado["edad"]["<= 23.0"]/contadores["edad"]["<= 23.0"]            
mean_result_agrupado["edad"]["> 40.22"] = mean_result_agrupado["edad"]["> 40.22"]/contadores["edad"]["> 40.22"]         
mean_result_agrupado["edad"]["(23.0, 40.22]"] = mean_result_agrupado["edad"]["(23.0, 40.22]"]/contadores["edad"]["(23.0, 40.22]"]          
mean_result_agrupado["genero"]["H"] = mean_result_agrupado["genero"]["H"]/contadores["genero"]["H"]         
mean_result_agrupado["genero"]["M"] = mean_result_agrupado["genero"]["M"]/contadores["genero"]["M"]         
mean_result_agrupado["experiencia"]["Si"] = mean_result_agrupado["experiencia"]["Si"]/contadores["experiencia"]["Si"]           
mean_result_agrupado["experiencia"]["No"] = mean_result_agrupado["experiencia"]["No"]/contadores["experiencia"]["No"] 

# Exportar a un diccionario
with open(PATH_VARIABLES+"/mean_result_agrupado.pkl", 'wb') as archivo:
    pickle.dump(mean_result_agrupado, archivo)
    

print(contadores)
mean_result_agrupado

{'genero': {'H': 36, 'M': 32}, 'experiencia': {'Si': 14, 'No': 54}, 'edad': {'<= 23.0': 23, '(23.0, 40.22]': 21, '> 40.22': 24}}


{'genero': {'H': array([[0.56481481, 1.85834004],
         [0.66666667, 1.94078555],
         [0.75925926, 1.87698734],
         [0.82407407, 1.75728743],
         [0.87962963, 1.64162557],
         [0.80555556, 1.49409094],
         [0.90740741, 1.36635745],
         [0.88888889, 1.55385186],
         [0.87962963, 1.36193869],
         [0.89814815, 1.29749699],
         [0.59259259, 1.9304733 ],
         [0.69444444, 1.6380961 ],
         [0.73148148, 1.55595377],
         [0.76851852, 1.81407645],
         [0.82407407, 1.45168319],
         [0.81481481, 1.45695439],
         [0.82407407, 1.50582994],
         [0.85185185, 1.31347368],
         [0.90740741, 1.34472268],
         [0.86111111, 1.34343535],
         [0.58333333, 2.15047196],
         [0.81481481, 2.14753613],
         [0.67592593, 2.00067933],
         [0.67592593, 1.85954808],
         [0.75925926, 1.61703923],
         [0.75925926, 1.72371358],
         [0.7962963 , 1.6924783 ],
         [0.86111111, 1.45409623],
     